# PyTorch Tutorial

- Notes from a [YT Tutorial](https://www.youtube.com/watch?v=GIsg-ZUy0MY)
- learn the basics and PyTorch's implementation of a linear regression here

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision
from torchvision import transforms, datasets

## 0. Basics - Tensors & Gradients

- Remember that Tensors are like scalars, vectors, or matrices
- They are floats in an n by n dimensional array

In [5]:
# like NumPy arrays

t0 = torch.tensor(3) # Scalar
t1 = torch.tensor([5, 3]) 
t2 = torch.tensor([2, 2])
t3 = torch.tensor([[5., 6], [7, 8], [9, 10]]) # Matrix

print(t1*t2, '\n') # elemement by element multiplication (5 * 2), (3 * 1)
print(t2+t3) # also element by element addition

tensor([10,  6]) 

tensor([[ 7.,  8.],
        [ 9., 10.],
        [11., 12.]])


In [6]:
# 3-dimensional array
t4 = torch.tensor([
    [[11, 12, 13], 
     [13, 14, 15]], 
    [[15, 16, 17], 
     [17, 18, 19.]]])
t4

tensor([[[11., 12., 13.],
         [13., 14., 15.]],

        [[15., 16., 17.],
         [17., 18., 19.]]])

In [7]:
t0.shape, t1.shape, t2.shape # for z 3 rows, 2 columns

(torch.Size([]), torch.Size([2]), torch.Size([2]))

In [8]:
t3.dtype # A float type

torch.float32

### Tensor operations & Gradients

We can combine tensors with the usual arithmetic operations.

- Note:
`torch.tensor` infers the dtype automatically, while `torch.Tensor` returns a torch.FloatTensor.
I would recommend to stick to `torch.tensor`, which also has arguments like dtype, if you would like to change the type.

In [9]:
# Create tensors
x = torch.tensor(3)
w = torch.tensor(4., requires_grad=True) # this is to allow for optimization and calculating derivatives (gradient = derivative)
b = torch.tensor(5., requires_grad=True)
x, w, b

(tensor(3), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [10]:
# Arithmetic operations
y = w * x + b 
y

tensor(17., grad_fn=<AddBackward0>)

As expected, y is a tensor with the value 3 * 4 + 5 = 17. What makes PyTorch unique is that we can automatically compute the derivative of y w.r.t. the tensors that have requires_grad set to True i.e. w and b. This feature of PyTorch is called autograd (automatic gradients).
To compute the derivatives, we can invoke the .backward method on our result y.

In [11]:
# Compute derivatives
y.backward()

In [12]:
# Display gradients (differentiation)
# y = w * x + b
#  dy/dw = w dropped, b reduced to 0, x remains
#  dy/db = b dropped, as well as w*x, so 1 remains

print('dy/dx:', x.grad) # No gradient
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


As expected, dy/dw has the same value as x, i.e., 3, and dy/db has the value 1. Note that x.grad is None because x doesn't have requires_grad set to True.
The "grad" in w.grad is short for gradient, which is another term for derivative. The term gradient is primarily used while dealing with vectors and matrices.

### Tensor functions

Apart from arithmetic operations, the torch module also contains many functions for creating and manipulating tensors. Let's look at some examples.

In [13]:
y = torch.rand([2, 5]) # for random numbers, 2 'rows', 5 'columns'
print(y)

tensor([[0.0310, 0.3794, 0.2847, 0.4553, 0.3774],
        [0.4007, 0.3795, 0.4146, 0.2424, 0.3370]])


In [14]:
# Flattens the tensor (provided you set 1 by n elements (10 in this case))

y.view([1, 10]) # important to fit in the model
y = y.view([1, 10]) # saves it in place
y

tensor([[0.0310, 0.3794, 0.2847, 0.4553, 0.3774, 0.4007, 0.3795, 0.4146, 0.2424,
         0.3370]])

In [15]:
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [16]:
# Create a tensor with a fixed value for every element
t6 = torch.full((3, 2), 42)
t6

tensor([[42, 42],
        [42, 42],
        [42, 42]])

In [17]:
# Concatenate two tensors with compatible shapes

t7 = torch.cat((t3, t6))
t7

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [42., 42.],
        [42., 42.],
        [42., 42.]])

In [18]:
# Compute the sin of each element
t8 = torch.sin(t7)
t8

tensor([[-0.9589, -0.2794],
        [ 0.6570,  0.9894],
        [ 0.4121, -0.5440],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165]])

In [19]:
# Change the shape of a tensor
t9 = t8.reshape(3, 2, 2)
t9

tensor([[[-0.9589, -0.2794],
         [ 0.6570,  0.9894]],

        [[ 0.4121, -0.5440],
         [-0.9165, -0.9165]],

        [[-0.9165, -0.9165],
         [-0.9165, -0.9165]]])

In [20]:
x = np.array([[1, 2], [3, 4]]) # Remember the double brackets
x

array([[1, 2],
       [3, 4]])

In [21]:
# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
y

tensor([[1, 2],
        [3, 4]])

In [22]:
print(type(x), type(y), '\n')
print(x.dtype, y.dtype)

<class 'numpy.ndarray'> <class 'torch.Tensor'> 

int64 torch.int64


In [23]:
# Convert back to a np array
z = y.numpy()
z

array([[1, 2],
       [3, 4]])

## 1. Linear Regression with PyTorch (and Gradient Descent)

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: Linear regression. We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall, and humidity (input variables or features) in a region. Here's the training data:

In [24]:
# Let's use some simple self-generated data
# temperature (C), rainfall (mm), humidity (%) (n rows, 3 columns)

temperature = torch.randint(5, 45, [50, 1]) # Create columns
rainfall = torch.randint(40, 150, [50, 1])
humidity = torch.randint(30, 70, [50, 1])

In [25]:
inputs = torch.cat((temperature, rainfall, humidity), 1) # Add columns
inputs[:5]

tensor([[ 13,  87,  37],
        [ 16, 121,  30],
        [ 32,  94,  44],
        [ 24, 112,  58],
        [ 17,  52,  47]])

In [26]:
inputs.dtype

torch.int64

LongTensor is synonymous with integer. PyTorch won't accept a FloatTensor as categorical target, so it's telling you to cast your tensor to LongTensor. This is how you should change your target dtype:



In [27]:
inputs = inputs.type(torch.FloatTensor)

In [28]:
# The same for your target outputs
# If using direct torch method, be wary of differentiating between LongTensor and FloatTensor

apples = torch.randint(20, 120, [50, 1])
oranges = torch.randint(35, 150, [50, 1])

targets = torch.cat((apples, oranges), 1)
targets[:5]

tensor([[ 89,  57],
        [ 63, 143],
        [100,  38],
        [106,  77],
        [ 33,  74]])

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :
- yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
- yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

- Think of wn = coefficient, and b1/b2 = intercept

In [29]:
# Weights and biases (randn allows randomized numbers between -1, 1, mean 0, stdev 1)
# Make sure inputs, weights and biases are consistent = FloatTensor or LongTensor (integer)

w = torch.randn(2, 3, requires_grad=True).type(torch.FloatTensor) # 2 targets (apples, oranges), 3 features
b = torch.randn(2, requires_grad=True).type(torch.FloatTensor) # 2 intercepts
print(w)
print(b)

tensor([[-0.0597,  1.4826,  1.2079],
        [-0.7456, -0.6224, -0.4537]], requires_grad=True)
tensor([1.4415, 1.7054], requires_grad=True)


In [30]:
# Create a manual model:
# @ represents matrix multiplication in PyTorch, and the .t method returns the transpose of a tensor.
# The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

def linreg(x):
    return x @ w.t() + b 

In [31]:
# Generate predictions
# This is an extremely shit model (- apples & oranges?)

preds = linreg(inputs)
print(preds[:10])

tensor([[ 174.3432,  -78.9229],
        [ 216.1175,  -99.1460],
        [ 192.0431, -100.6227],
        [ 236.1173, -112.2122],
        [ 134.2924,  -64.6580],
        [ 140.9667,  -70.9395],
        [ 139.3103,  -64.3227],
        [ 231.2651, -112.9376],
        [ 191.7912, -106.2322],
        [ 180.2758,  -89.7309]], grad_fn=<SliceBackward>)


### Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:
- Calculate the difference between the two matrices (preds and targets).
- Square all elements of the difference matrix to remove negative values.
- Calculate the average of the elements in the resulting matrix.


The result is a single number, known as the mean squared error (MSE).

In [32]:
diff = preds - targets
diff[:10]

tensor([[  85.3432, -135.9229],
        [ 153.1175, -242.1460],
        [  92.0431, -138.6227],
        [ 130.1173, -189.2122],
        [ 101.2924, -138.6580],
        [ 103.9667, -131.9395],
        [ 116.3103, -212.3227],
        [ 149.2651, -235.9376],
        [  86.7912, -197.2322],
        [  98.2758, -189.7309]], grad_fn=<SliceBackward>)

In [33]:
# MSE
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel() # .numel returns the number of elements in a tensor

In [34]:
# Compute loss - you want this to be as low as possible
loss = mse(preds, targets)
print(loss)

tensor(28447.0957, grad_fn=<DivBackward0>)


### Compute Gradients

- With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have requires_grad set to True.

In [35]:
# Compute gradients
loss.backward()

The gradients are stored in the .grad property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [36]:
# Gradients for weights & biases
# The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest.

print(w)
print(w.grad, "\n")
print(b)
print(b.grad)

tensor([[-0.0597,  1.4826,  1.2079],
        [-0.7456, -0.6224, -0.4537]], requires_grad=True)
tensor([[  2871.0332,  12735.9453,   6386.1592],
        [ -4614.7373, -17992.8672,  -9587.8135]]) 

tensor([1.4415, 1.7054], requires_grad=True)
tensor([ 125.8641, -190.9120])


As the gradients in both b & w are negative, we want to optimize this by increasing their values, and vice versa if the reverse

We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount.

We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the learning rate of the algorithm.


We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [37]:
# Adjust weights & reset gradients:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [38]:
# They all should be lower now
print(w)
print(w.grad, "\n")
print(b)
print(b.grad)

tensor([[-0.0884,  1.3552,  1.1440],
        [-0.6995, -0.4425, -0.3578]], requires_grad=True)
tensor([[0., 0., 0.],
        [0., 0., 0.]]) 

tensor([1.4403, 1.7073], requires_grad=True)
tensor([0., 0.])


In [39]:
preds2 = linreg(inputs)

In [40]:
# Let's verify that the loss is actually lower
loss = mse(preds2, targets)
print(loss)

tensor(22354.4004, grad_fn=<DivBackward0>)


### Train the model using gradient descent
As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:
- Generate predictions
- Calculate the loss
- Compute gradients w.r.t the weights and biases
- Adjust the weights by subtracting a small quantity proportional to the gradient
- Reset the gradients to zero

In [41]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[  2500.8784,  11215.8789,   5606.4473],
        [ -4081.2861, -15807.6426,  -8464.7686]])
tensor([ 110.3893, -168.6348])


In [42]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [43]:
print(w)
print(b)

tensor([[-0.1134,  1.2431,  1.0879],
        [-0.6587, -0.2844, -0.2731]], requires_grad=True)
tensor([1.4392, 1.7090], requires_grad=True)


In [44]:
# Calculate loss
preds3 = linreg(inputs)
loss = mse(preds3, targets)
print(loss)

tensor(17638.9023, grad_fn=<DivBackward0>)


### Train for multiple epochs
To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [45]:
# Train for 100 epochs
# You can rerun this and calculate loss
for i in range(1000):
    preds = linreg(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5 # The learning rate, keep this number very small
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [46]:
# Calculate loss
preds = linreg(inputs)
loss = mse(preds, targets)
print(loss)

tensor(1147.0916, grad_fn=<DivBackward0>)


In [47]:
preds[:10]

tensor([[ 50.4503,  75.0393],
        [ 51.9040,  81.5913],
        [ 70.5128,  90.8103],
        [ 77.6444, 108.8123],
        [ 56.2272,  73.3960],
        [ 65.0160,  81.6422],
        [ 46.0013,  64.9196],
        [ 84.0145, 113.1495],
        [ 71.7592,  88.9563],
        [ 60.9973,  82.3617]], grad_fn=<SliceBackward>)

In [48]:
targets[:10]

tensor([[ 89,  57],
        [ 63, 143],
        [100,  38],
        [106,  77],
        [ 33,  74],
        [ 37,  61],
        [ 23, 148],
        [ 82, 123],
        [105,  91],
        [ 82, 100]])

#### The more conventional built-in method

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.


Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [49]:
import torch.nn as nn

In [50]:
# Use new inputs and targets (with numpy, then conversion this time)

In [51]:
temp = np.random.randint(5, 45, [50, 1])
rain = np.random.randint(40, 150, [50, 1])
humid = np.random.randint(30, 70, [50, 1])

In [52]:
inputs = np.append(temp, np.append(rain, humid, axis=1), axis=1)
inputs[:5]

array([[ 30, 127,  46],
       [ 31,  99,  58],
       [ 33,  90,  61],
       [ 40,  48,  51],
       [ 43,  57,  32]])

In [53]:
apples = np.random.randint(20, 120, [50, 1])
oranges = np.random.randint(35, 150, [50, 1])

targets = np.append(apples, oranges, axis=1)
targets[:5]

array([[ 93,  53],
       [ 37,  75],
       [ 54,  66],
       [106,  80],
       [ 94,  63]])

In [54]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [55]:
inputs = inputs.type(torch.FloatTensor)
targets = targets.type(torch.FloatTensor)

### Dataset and DataLoader
We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [56]:
from torch.utils.data import TensorDataset

In [57]:
# Define and preprocess dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 30., 127.,  46.],
         [ 31.,  99.,  58.],
         [ 33.,  90.,  61.]]),
 tensor([[93., 53.],
         [37., 75.],
         [54., 66.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.


We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [58]:
from torch.utils.data import DataLoader

In [59]:
# Define data loader
# Batch sizes more important with lots of data

batch_size = 10 # 20% of dataset
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [60]:
# To view each batch (& verify size and shuffled)
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 14.,  55.,  47.],
        [ 12., 133.,  50.],
        [ 20.,  97.,  54.],
        [ 15., 130.,  41.],
        [ 43.,  47.,  59.],
        [ 43., 123.,  46.],
        [ 20.,  52.,  63.],
        [ 24., 109.,  42.],
        [ 38.,  80.,  58.],
        [ 44.,  64.,  37.]])
tensor([[ 56.,  83.],
        [ 89.,  35.],
        [ 46.,  42.],
        [ 63.,  94.],
        [109.,  84.],
        [ 43., 136.],
        [ 27.,  75.],
        [ 86., 117.],
        [113.,  43.],
        [ 38., 120.]])


#### nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [61]:
# Define model --> nn.Linear(input, output)

model = nn.Linear(3, 2) # 3 features (weights), 2 outputs (intercepts)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3902, -0.3410, -0.1206],
        [-0.0979, -0.0052, -0.3672]], requires_grad=True)
Parameter containing:
tensor([0.4604, 0.5738], requires_grad=True)


In [62]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.3902, -0.3410, -0.1206],
         [-0.0979, -0.0052, -0.3672]], requires_grad=True),
 Parameter containing:
 tensor([0.4604, 0.5738], requires_grad=True)]

In [63]:
# Generate predictions
# Still bad
preds = model(inputs)
preds[:5]

tensor([[-60.0991, -19.9202],
        [-52.3889, -24.2774],
        [-50.4622, -25.5275],
        [-37.6653, -22.3210],
        [-39.6127, -15.6857]], grad_fn=<SliceBackward>)

#### Loss Function
Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [64]:
# Import nn.functional - similar to sklearn metrics or utils
import torch.nn.functional as F

In [65]:
# Get loss - save as a variable
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets).type(torch.FloatTensor)
print(loss)
print(loss.dtype)

tensor(13823.3037, grad_fn=<MseLossBackward>)
torch.float32


#### Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.

In [66]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

### Train the model
We are now ready to train the model. We'll follow the same process as above to implement gradient descent


The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.

In [67]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            loss = loss.type(torch.FloatTensor)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [74]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 1312.9205
Epoch [20/100], Loss: 1765.7009
Epoch [30/100], Loss: 800.1871
Epoch [40/100], Loss: 1184.6575
Epoch [50/100], Loss: 1390.1193
Epoch [60/100], Loss: 1565.3336
Epoch [70/100], Loss: 1093.3812
Epoch [80/100], Loss: 1469.9753
Epoch [90/100], Loss: 1635.4625
Epoch [100/100], Loss: 1441.4415


In [75]:
# Generate predictions
preds = model(inputs)
preds[:5]

tensor([[81.9067, 93.0223],
        [76.4611, 91.6100],
        [75.1759, 90.7977],
        [58.4788, 67.5909],
        [55.7836, 56.7058]], grad_fn=<SliceBackward>)

In [76]:
# Compare with targets
targets[:5]

tensor([[ 93.,  53.],
        [ 37.,  75.],
        [ 54.,  66.],
        [106.,  80.],
        [ 94.,  63.]])